In [10]:
with open("inputs/Day_17.txt") as f:
    raw_input_data = f.read()

In [30]:
from collections import namedtuple, defaultdict

    
def part_1_solution(raw_input):
    puzzle_input = list(map(int, raw_input.split(',')))
    
    raw_grid = get_raw_grid(puzzle_input)
    grid = parse_grid(raw_grid)
    
    alignment_sum = 0
    
    for block_position in grid:
        if is_intersection(block_position, grid):
            alignment_sum += block_position[0] * block_position[1]
    
    return alignment_sum


def get_raw_grid(program):
    outputs = list()
    computer = start_computer(program)
    item = next(computer)
    
    while item is not Finished:
        if isinstance(item, Output):
            char = chr(item.code)
            outputs.append(char)
                    
            item = next(computer)
        else:
            raise Exception(f"Unsuported item from generator: {item}")
            
    return ''.join(outputs)


def parse_grid(raw_grid):
    grid = set()
    
    for y, row in enumerate(raw_grid.split('\n')):
        for x, obj in enumerate(row):
            if obj != '.':
                block_position = (x, y)
                grid.add(block_position)
            
    return grid


def is_intersection(block_position, grid):
    for x_offset, y_offset in ((0, 1), (1, 0), (0, -1), (-1, 0)):
        canditdate_position = (block_position[0] + x_offset, block_position[1] + y_offset)

        if canditdate_position not in grid:
            return False
            
    return True

Finished = object()
Input = object()
Output = namedtuple('Output', ('code'))


def start_computer(sequence):
    index = 0
    relative_base = 0
    memory = defaultdict(int)
    
    for i, value in enumerate(sequence):
        memory[i] = value
        
    diag_nbr = None
    while True:
        opt_code = memory[index]
        opt_code_with_modes = str(opt_code).zfill(5)
        opt_code = int(opt_code_with_modes[-2:])
        modes = opt_code_with_modes[:-2]
        
        
        par_1_address = get_parameter_address(memory, modes[-1], index + 1, relative_base)
        par_2_address = get_parameter_address(memory, modes[-2], index + 2, relative_base)
        par_3_address = get_parameter_address(memory, modes[-3], index + 3, relative_base)
        
        par_1 = memory[par_1_address]
        par_2 = memory[par_2_address]
        
        if opt_code == 99:
            break
        elif opt_code == 3:
            target_address = par_1_address
            memory[target_address] = yield Input
            index += 2
        elif opt_code == 4:
            diag_nbr = par_1
            yield Output(diag_nbr)
                
            index += 2
        elif opt_code == 1:
            target_address = par_3_address
            memory[target_address] = par_1 + par_2
            index += 4
        elif opt_code == 2:
            target_address = par_3_address
            memory[target_address] = par_1 * par_2
            index += 4
        elif opt_code == 5:
            if par_1 != 0:
                index = par_2
            else:
                index += 3
        elif opt_code == 6:
            if par_1 == 0:
                index = par_2
            else:
                index += 3
        elif opt_code == 7:
            target_address = par_3_address
            memory[target_address] = 1 if par_1 < par_2 else 0
            index += 4
        elif opt_code == 8:
            target_address = par_3_address
            memory[target_address] = 1 if par_1 == par_2 else 0
            index += 4
        elif opt_code == 9:
            relative_base += par_1
            index += 2
        else:
            print(f"Wrong code: {opt_code}")  
    
    yield Finished

            
def get_parameter_address(memory, mode, par_index, relative_base):
    if mode == "0":
        return memory[par_index]
    elif mode == "1":
        return par_index
    elif mode == "2":
        return relative_base + memory[par_index]
    else:
        print(f"[ERROR] Wrong mode code: {mode}")

In [31]:
print(f"Part 1 solution: {part_1_solution(raw_input_data)}")

Part 1 solution: 6052


In [64]:
from collections import namedtuple, defaultdict
from enum import Enum


class Orientation(Enum):
    Up = (0, -1)
    Down = (0, 1)
    Right = (1, 0)
    Left = (-1, 0)

    
def part_2_solution(raw_input):
    puzzle_input = list(map(int, raw_input.split(',')))
    
    raw_grid = get_raw_grid(puzzle_input)
    grid, robot_position, robot_orientation = parse_grid(raw_grid)
    moves = get_moves_list(grid, robot_position, robot_orientation)
    
    return ','.join(moves)
    
    
def get_moves_list(grid, robot_position, robot_orientation):
    moves = list()
    current_direction = robot_orientation
    current_position = robot_position

    while True:
        current_direction, turn_mark = get_next_direction(current_position, current_direction, grid)
        
        if current_direction is End:
            return moves
        
        moves.append(turn_mark)
        
        next_position = move(current_position, current_direction)
        moves_cnt = 0
        
        while next_position in grid:
            current_position = next_position
            moves_cnt += 1
            
            next_position = move(current_position, current_direction)
        
        moves.append(str(moves_cnt))
        
    
right_transition = {
    Orientation.Up: Orientation.Right,
    Orientation.Right: Orientation.Down,
    Orientation.Down: Orientation.Left,
    Orientation.Left: Orientation.Up
}

left_transition = {v: k for k, v in right_transition.items()}

End = object()

def get_next_direction(position, old_direction, grid):
    # try to turn right
    direction_candidate = right_transition[old_direction]
    new_position = move(position, direction_candidate)
    
    if new_position in grid:
        return direction_candidate, 'R'
    
    # try to turn left
    direction_candidate = left_transition[old_direction]
    new_position = move(position, direction_candidate)
    
    if new_position in grid:
        return direction_candidate, 'L'
    
    return End, ''
    
    
def move(position, direction):
    return position[0] + direction.value[0], position[1] + direction.value[1]

def get_raw_grid(program):
    outputs = list()
    computer = start_computer(program)
    item = next(computer)
    
    while item is not Finished:
        if isinstance(item, Output):
            char = chr(item.code)
            outputs.append(char)
                    
            item = next(computer)
        else:
            raise Exception(f"Unsuported item from generator: {item}")
            
    return ''.join(outputs)


def parse_grid(raw_grid):
    grid = set()
    robot_orientation = None
    
    for y, row in enumerate(raw_grid.split('\n')):
        for x, obj in enumerate(row):
            block_position = (x, y)
            
            if obj != '.':
                grid.add(block_position)
            
            if obj == '^':
                robot_orientation = Orientation.Up
                robot_position = block_position
            elif obj == 'v':
                robot_orientation = Orientation.Down
                robot_position = block_position
            elif obj == '<':
                robot_orientation = Orientation.Left
                robot_position = block_position
            elif obj == '>':
                robot_orientation = Orientation.Right
                robot_position = block_position
            
    return grid, robot_position, robot_orientation

    
    
Finished = object()
Input = object()
Output = namedtuple('Output', ('code'))


def start_computer(sequence):
    index = 0
    relative_base = 0
    memory = defaultdict(int)
    
    for i, value in enumerate(sequence):
        memory[i] = value
        
    diag_nbr = None
    while True:
        opt_code = memory[index]
        opt_code_with_modes = str(opt_code).zfill(5)
        opt_code = int(opt_code_with_modes[-2:])
        modes = opt_code_with_modes[:-2]
        
        
        par_1_address = get_parameter_address(memory, modes[-1], index + 1, relative_base)
        par_2_address = get_parameter_address(memory, modes[-2], index + 2, relative_base)
        par_3_address = get_parameter_address(memory, modes[-3], index + 3, relative_base)
        
        par_1 = memory[par_1_address]
        par_2 = memory[par_2_address]
        
        if opt_code == 99:
            break
        elif opt_code == 3:
            target_address = par_1_address
            memory[target_address] = yield Input
            index += 2
        elif opt_code == 4:
            diag_nbr = par_1
            yield Output(diag_nbr)
                
            index += 2
        elif opt_code == 1:
            target_address = par_3_address
            memory[target_address] = par_1 + par_2
            index += 4
        elif opt_code == 2:
            target_address = par_3_address
            memory[target_address] = par_1 * par_2
            index += 4
        elif opt_code == 5:
            if par_1 != 0:
                index = par_2
            else:
                index += 3
        elif opt_code == 6:
            if par_1 == 0:
                index = par_2
            else:
                index += 3
        elif opt_code == 7:
            target_address = par_3_address
            memory[target_address] = 1 if par_1 < par_2 else 0
            index += 4
        elif opt_code == 8:
            target_address = par_3_address
            memory[target_address] = 1 if par_1 == par_2 else 0
            index += 4
        elif opt_code == 9:
            relative_base += par_1
            index += 2
        else:
            print(f"Wrong code: {opt_code}")  
    
    yield Finished

            
def get_parameter_address(memory, mode, par_index, relative_base):
    if mode == "0":
        return memory[par_index]
    elif mode == "1":
        return par_index
    elif mode == "2":
        return relative_base + memory[par_index]
    else:
        print(f"[ERROR] Wrong mode code: {mode}")

In [57]:
print(f"Part 2 solution: {part_2_solution(raw_input_data)}")

Part 2 solution: L,6,R,12,L,4,L,6,R,6,L,6,R,12,R,6,L,6,R,12,L,6,L,10,L,10,R,6,L,6,R,12,L,4,L,6,R,6,L,6,R,12,L,6,L,10,L,10,R,6,L,6,R,12,L,4,L,6,R,6,L,6,R,12,L,6,L,10,L,10,R,6


In [113]:
def get_collected_dust(routines, raw_puzzle_input):
    puzzle_input = list(map(int, raw_puzzle_input.split(',')))
    puzzle_input[0] = 2
    
    robot = start_computer(puzzle_input)
    routine_idx = 0
    item = next(robot)

    while item is not Finished: 
        if isinstance(item, Output):
            if item.code > 255:
                collected_dust = item.code 
#             parsed_output = chr(item.code) if item.code <= 255 else item.code
#             print(parsed_output, end="")
            item = next(robot)
        elif item is Input:
            if routine_idx >= len(routines):
                item = robot.send(ord('n'))
                item = robot.send(10)
                continue
            
            for char in routines[routine_idx]:
                item = robot.send(ord(char))
            item = robot.send(10)
            routine_idx += 1

    return collected_dust

In [115]:
movement_routine = "A,B,B,C,A,B,C,A,B,C"
a_routine = "L,6,R,12,L,4,L,6"
b_routine = "R,6,L,6,R,12"
c_routine = "L,6,L,10,L,10,R,6"
routines = [movement_routine, a_routine, b_routine, c_routine]
print(f"Part 2 solution: {get_collected_dust(routines, raw_input_data)}")

Part 2 solution: 752491
